In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
def get_metrics_value(algorithm_name, func_ind, ndim, runs=51, year=2017):
    file_name = f"result/{algorithm_name}/F{func_ind}{year}-{ndim}-{runs}.csv"
    df = pd.read_csv(file_name)
    last_non_nan_values = df.ffill().iloc[-1]
    metrics_value = last_non_nan_values.describe()
    metrics_value = [algorithm_name] + [metrics_value[i] for i in ["min", "50%", "mean", "max", "std"]]
    return metrics_value

In [3]:
# 实验结果展示
ndim = 100
func_ind = 1

metrics_df = [get_metrics_value(algorithm_name, func_ind, ndim) for algorithm_name in os.listdir("result")]
metrics_df = pd.DataFrame(metrics_df, columns=["name", "min", "50%", "mean", "max", "std"]).sort_values(by=["min", "std"])
metrics_df.reset_index(drop=True, inplace=True)
print(f"F* -> {func_ind * 100}")
metrics_df

F* -> 100


,name,min,50%,mean,max,std
0,L-SHADE,1.677528e+08,4.926174e+08,6.031544e+08,2.518070e+09,4.379058e+08
1,JADE,3.999394e+08,7.383079e+08,8.437596e+08,2.294172e+09,3.704557e+08
2,JADE-Blossoms,7.281653e+09,1.104285e+10,1.184252e+10,1.922352e+10,2.772394e+09
3,L-SHADE-Blossoms,1.064232e+10,1.994596e+10,2.069203e+10,3.225251e+10,5.387959e+09
4,Graph-Blossoms,1.093478e+10,1.872133e+10,1.873438e+10,3.838936e+10,4.490754e+09
5,Chain-Blossoms,2.405763e+10,3.563424e+10,3.524649e+10,4.691403e+10,5.113065e+09
6,PSO,3.866790e+10,4.958440e+10,5.136615e+10,7.402962e+10,9.138218e+09
7,AGSK,5.527153e+10,1.719930e+11,1.715232e+11,3.236901e+11,6.534272e+10
8,PSO-Blossoms,7.300046e+10,8.662343e+10,8.612021e+10,9.955207e+10,6.067459e+09
9,SSA,8.507582e+10,1.175728e+11,1.183545e+11,1.413740e+11,1.110708e+10


In [5]:
# 根据下面这篇文章提出的方法计算各算法的得分
# Awad NH, Ali MZ, Liang JJ, Qu BY, Suganthan PN (2016) Problem defnitions and evaluation criteria for the CEC 2017 special 
# session and competition on single objective bound constrained real-parameter numerical optimization. In: Technical Report, 
# Nanyang Technological University Singapore
metrics_dict = dict()
for algorithm_name in tqdm(os.listdir("result")):
    ef29 = []
    for ndim in [10, 30, 50, 100]:
        ef29_ndim = []
        for func_ind in range(1, 30):
            file_name = f"result/{algorithm_name}/F{func_ind}2017-{ndim}-51.csv"
            df = pd.read_csv(file_name)
            last_non_nan_values = df.ffill().iloc[-1]
            last_non_nan_values = last_non_nan_values - func_ind * 100
            assert all(last_non_nan_values >= 0)
            metrics_value = last_non_nan_values.describe()
            metrics_value = [metrics_value[i] for i in ["min", "50%", "mean", "max", "std"]]
            ef29_ndim.append(metrics_value)
        ef29.append(ef29_ndim)
    metrics_dict[algorithm_name] = np.array(ef29) # 4 * 29 * 5
metrics_dict.keys()

100%|██████████| 20/20 [00:24<00:00,  1.21s/it]


dict_keys(['AGSK', 'AGSK-Blossoms', 'Chain-Blossoms', 'DBO', 'DBO-Blossoms', 'Graph-Blossoms', 'GSK', 'GSK-Blossoms', 'GWO', 'GWO-Blossoms', 'HO', 'HO-Blossoms', 'JADE', 'JADE-Blossoms', 'L-SHADE', 'L-SHADE-Blossoms', 'PSO', 'PSO-Blossoms', 'SSA', 'SSA-Blossoms'])

In [6]:
weights = np.array([0.1, 0.2, 0.3, 0.4]).reshape(1, 4)
SE = np.vstack([weights @ v.sum(axis=1) for v in metrics_dict.values()])
Score1 = (1 - (SE - SE.min(axis=0)) / SE) * 50 # n * 5
SR = np.stack([v for v in metrics_dict.values()], axis=1).argsort(axis=1).sum(axis=2) # 4 * n * 5
SR = np.vstack([weights @ SR[:, i, :] for i in range(SR.shape[1])])
Score2 = (1 - (SR - SR.min(axis=0)) / SR) * 50 # n * 5
Score = Score1 + Score2

In [7]:
score_df = pd.DataFrame(Score, columns=["min", "50%", "mean", "max", "std"])
score_df["score_mean"] = score_df.mean(axis=1)
# score_df.insert(0, "algorithm", metrics_dict.keys())
score_df.index = metrics_dict.keys()
score_df.sort_values(by="score_mean", ascending=False, inplace=True)
score_df.style.highlight_max(axis=0, color="red")

,min,50%,mean,max,std,score_mean
L-SHADE,83.453961,89.206809,84.499235,77.812159,84.319621,83.858357
JADE,50.413097,73.188058,68.690969,83.144834,85.271041,72.141600
Graph-Blossoms,32.158236,40.857608,50.819847,51.269548,43.897365,43.800521
HO,42.950646,50.001805,41.514192,29.832387,43.472811,41.554368
SSA,35.111107,46.538761,39.247237,34.217396,46.304651,40.283831
JADE-Blossoms,36.096151,45.076277,39.874120,35.766032,43.452965,40.053109
GWO-Blossoms,27.540580,47.126693,39.018595,38.683547,39.013809,38.276645
SSA-Blossoms,30.770754,41.795527,35.771022,31.388752,50.000073,37.945225
HO-Blossoms,30.909370,45.810867,38.274275,32.359329,41.942483,37.859265
GSK-Blossoms,50.000005,44.805466,32.734859,25.164693,36.213843,37.783773
